<a href="https://colab.research.google.com/github/marceloasr94/Coursera_Capstone/blob/main/Coursera_Project_Week_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction:

The city of Rio de Janeiro/Brazil is very big and has more than 150 "neighborhood", if one is moving from an specific neighborhood in São Paulo to Rio de Janeiro and wants to live in a neighborhood that at least looks like the one she/he lived before, where should they search places?

##Problem:

One is moving from the neighborhood "Moema", in São Paulo and never been in Rio de Janeiro before, this person works and study "Home Office" so he/she doesn't care for work/university distance. Which neighborhoods he/she should search in Rio de Janeiro? Which are the most looks like neighborhoods?
So We will separate neighborhood by Cluster and compare which cluster has more "Common Venues" to Moema, São Paulo. With this one will have a leaner list of neighborhoods to look.

##Colab Code:

If the rendering of Maps are not cool in Git, you may see it directly using my Colab:

https://colab.research.google.com/drive/1r45IfNMnfyHsqoUCpYDfZeGxr_aH4xGm?usp=sharing

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 2.2MB/s 


In [3]:
import geocoder
from geopy.geocoders import Nominatim

Today the person Lives in Moema, a Neighborhood in São Paulo:

In [4]:
address = 'Moema, Sao Paulo, SP'

label="Moema"
geolocator = Nominatim(user_agent="project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Moema/SP are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Moema/SP are -23.597085, -46.6628884.


In [5]:
mapa_sp=folium.Map(location=[latitude, longitude], zoom_start=15)
moema=folium.map.FeatureGroup()
moema.add_child(folium.CircleMarker(
            [latitude, longitude],
            radius=200,
            popup=label,
            color='blue',
            fill_color='#3186cc',
            fill_opacity=0.4))
mapa_sp.add_child(moema)      
mapa_sp

In [6]:
col=['Neighbourhood', 'Latitude', 'Longitude']
data=[[label, latitude, longitude]]
sp_df = pd.DataFrame(data, columns=col)
sp_df


,Neighbourhood,Latitude,Longitude
0,Moema,-23.597085,-46.662888


The city we will be studying is Rio de Janeiro, let's get the Data from a Wikipedia page. The Page classifies each neighborhood by HDI and has a lot of columns that does not matter to us, let's get rid of them:

In [7]:
pd.set_option('display.max_rows', 140)
rj = pd.read_html("https://pt.wikipedia.org/wiki/Lista_de_bairros_do_Rio_de_Janeiro_por_IDH",header=1)[0]
rj = rj.drop(rj.columns[[0,2,3,4,5,6,7,8,9]], axis=1)
rj=rj.rename(columns={"Bairro ou grupo de bairros":"Neighbourhood"})
rj

,Neighbourhood
0,"Barra da Tijuca, Joá"
1,Gávea
2,Leblon
3,Jardim Guanabara
4,Ipanema
5,Lagoa
6,Flamengo
7,Humaitá
8,Laranjeiras
9,Jardim Botânico


In [8]:
rj.dtypes

Neighbourhood    object
dtype: object

As our data originated from a Wikipedia page that classifies each neighbourhood by its human development index(HDI), some of the neighbourhoods has the same position, so they are at the same ROW, separeted by commas, let's explode it...
Ex: In Row 7 I Have "Barra da Tijuca" and "Joá"

In [9]:
#Função para Separar Linhas com ","
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [10]:
new_rj=splitDataFrameList(rj,"Neighbourhood",",")

In [11]:
pd.set_option('display.max_rows', 165)
new_rj=new_rj.drop([76,99,110,122])
new_rj=new_rj.reset_index()

In [12]:
new_rj=new_rj.drop("index", axis=1)
new_rj

,Neighbourhood
0,Barra da Tijuca
1,Joá
2,Gávea
3,Leblon
4,Jardim Guanabara
5,Ipanema
6,Lagoa
7,Flamengo
8,Humaitá
9,Laranjeiras


Now I may Capture each Neighbourhood Longitude and Latitude using Geolocator:

In [13]:
new_rj['Latitude']=0.1
new_rj['Longitude']=0.1
for i,label in enumerate(new_rj['Neighbourhood']):
  geolocator = Nominatim(user_agent="project")
  location = geolocator.geocode(label+", Rio de Janeiro,RJ")
  latitude = location.latitude
  longitude = location.longitude
  new_rj['Latitude'][i]=latitude
  new_rj['Longitude'][i]=longitude
  print('The',i,' Lat and Long are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


The 0  Lat and Long are -22.9997404, -43.3659929.
The 1  Lat and Long are -23.0077797, -43.2872344.
The 2  Lat and Long are -22.9814243, -43.2383245.
The 3  Lat and Long are -22.983556, -43.2249377.
The 4  Lat and Long are -22.8128362, -43.2007792.
The 5  Lat and Long are -22.9839557, -43.2022163.
The 6  Lat and Long are -22.9624658, -43.2024884.
The 7  Lat and Long are -22.9339841, -43.1745744.
The 8  Lat and Long are -22.9546413, -43.2004797.
The 9  Lat and Long are -22.9343173, -43.1878165.
The 10  Lat and Long are -22.968384550000003, -43.22869449858106.
The 11  Lat and Long are -22.9719642, -43.1843432.
The 12  Lat and Long are -22.961704, -43.1669042.
The 13  Lat and Long are -22.9515096, -43.1862969.
The 14  Lat and Long are -22.954074, -43.1679727.
The 15  Lat and Long are -22.912091949999997, -43.23114540590559.
The 16  Lat and Long are -22.902953, -43.5591295.
The 17  Lat and Long are -22.4523313, -43.1350743.
The 18  Lat and Long are -22.9026425, -43.2817263.
The 19  Lat and

Finally my Study Data will be:

In [14]:
new_rj

,Neighbourhood,Latitude,Longitude
0,Barra da Tijuca,-22.999740,-43.365993
1,Joá,-23.007780,-43.287234
2,Gávea,-22.981424,-43.238324
3,Leblon,-22.983556,-43.224938
4,Jardim Guanabara,-22.812836,-43.200779
5,Ipanema,-22.983956,-43.202216
6,Lagoa,-22.962466,-43.202488
7,Flamengo,-22.933984,-43.174574
8,Humaitá,-22.954641,-43.200480
9,Laranjeiras,-22.934317,-43.187816


In [15]:
address_rj = 'Rio de Janeiro, RJ'

label="RJ"
geolocator = Nominatim(user_agent="project")
location_rj = geolocator.geocode(address_rj)
latitude_rj = location_rj.latitude
longitude_rj = location_rj.longitude
print('The geograpical coordinate of Rio De Janeiro/RJ are {}, {}.'.format(latitude_rj, longitude_rj))


The geograpical coordinate of Rio De Janeiro/RJ are -22.9110137, -43.2093727.


I May Plot each Neighbourhood in the map:

In [16]:
map_rj = folium.Map(location=[latitude_rj, longitude_rj], zoom_start=11)
# add markers to map
for lat, lng, label in zip(new_rj['Latitude'].values, new_rj['Longitude'].values, new_rj['Neighbourhood'].values):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rj)  
    
map_rj

## Week 5 - Clustering 

Now, let's Check which are the most Commons venues in Moema, São  Paulo so we can  compare to ours neighbourhoods clusters in RIO DE JANEIRO!

In [17]:


CLIENT_ID = '3MDKB1MXM5DDAKDADA3OQJZGZLW2K3HX41DUBXIGPELZJJOD' # your Foursquare ID
CLIENT_SECRET = 'MWJ0OIVBDKCJZ5OHXILELQNMIM0INEDTPJCA2W3N0ZR3JWJG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3MDKB1MXM5DDAKDADA3OQJZGZLW2K3HX41DUBXIGPELZJJOD
CLIENT_SECRET:MWJ0OIVBDKCJZ5OHXILELQNMIM0INEDTPJCA2W3N0ZR3JWJG


In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
sp_venues = getNearbyVenues(names=sp_df['Neighbourhood'],
                                   latitudes=sp_df['Latitude'],
                                   longitudes=sp_df['Longitude']
                                  )

Moema


In [21]:
sp_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Moema,-23.597085,-46.662888,Djapa Restaurante,-23.598476,-46.662900,Sushi Restaurant
1,Moema,-23.597085,-46.662888,Dell Anno Atelier,-23.595940,-46.663804,Furniture / Home Store
2,Moema,-23.597085,-46.662888,Pé no Parque,-23.597642,-46.665242,Snack Place
3,Moema,-23.597085,-46.662888,Glitz Mania,-23.599009,-46.661560,Salon / Barbershop
4,Moema,-23.597085,-46.662888,Royal Enfield Store,-23.597950,-46.661710,Motorcycle Shop
5,Moema,-23.597085,-46.662888,Panvel,-23.597149,-46.662910,Pharmacy
6,Moema,-23.597085,-46.662888,Mambo Supermercado,-23.599691,-46.665532,Supermarket
7,Moema,-23.597085,-46.662888,Au Au Mania,-23.598834,-46.665686,Pet Store
8,Moema,-23.597085,-46.662888,Millenium Livraria & Cafe,-23.599663,-46.661080,Bookstore
9,Moema,-23.597085,-46.662888,Arte de Viver,-23.596409,-46.666665,Yoga Studio


In [22]:
sp_onehot = pd.get_dummies(sp_venues[['Venue Category']], prefix="", prefix_sep="")
sp_onehot['Neighbourhood'] = sp_venues['Neighbourhood'] 
first = [sp_onehot.columns[-1]] + list(sp_onehot.columns[:-1])
sp_onehot = sp_onehot[first]
sp_grouped = sp_onehot.groupby('Neighbourhood').mean().reset_index()
sp_grouped

,Neighbourhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bike Rental / Bike Share,Bookstore,Burger Joint,Café,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Fast Food Restaurant,Fish & Chips Shop,Food,Furniture / Home Store,Greek Restaurant,Gym / Fitness Center,Gym Pool,Health Food Store,Hotel,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Music Venue,Park,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Plaza,Rest Area,Restaurant,Salon / Barbershop,Snack Place,Soccer Field,Spa,Supermarket,Sushi Restaurant,Thrift / Vintage Store,Trail,Whisky Bar,Wine Shop,Women's Store,Yoga Studio
0,Moema,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.027027,0.040541,0.013514,0.013514,0.013514,0.054054,0.013514,0.013514,0.013514,0.013514,0.013514,0.027027,0.040541,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.027027,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.027027,0.013514


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_sp_sorted = pd.DataFrame(columns=columns)
neighborhoods_sp_sorted['Neighbourhood'] = sp_grouped['Neighbourhood']

for ind in np.arange(sp_grouped.shape[0]):
    neighborhoods_sp_sorted.iloc[ind, 1:] = return_most_common_venues(sp_grouped.iloc[ind, :], num_top_venues)

neighborhoods_sp_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Moema,Furniture / Home Store,Dessert Shop,Japanese Restaurant,Snack Place,Pharmacy,Pizza Place,Plaza,Middle Eastern Restaurant,Restaurant,Burger Joint


In [25]:
rj_venues = getNearbyVenues(names=new_rj['Neighbourhood'],
                                   latitudes=new_rj['Latitude'],
                                   longitudes=new_rj['Longitude'])

Barra da Tijuca
 Joá
Gávea
Leblon
Jardim Guanabara
Ipanema
Lagoa
Flamengo
Humaitá
Laranjeiras
Jardim Botânico
Copacabana
Leme
Botafogo
 Urca
Maracanã
Campo Grande
Glória
Méier
Tijuca
 Alto da Boa Vista
Todos os Santos
Anil
Vila da Penha
Andaraí
Riachuelo
Campinho
 Vila Valqueire
Moneró
 Portuguesa
Catete
Vila Isabel
Cachambi
Pechincha
Freguesia
Recreio dos Bandeirantes
 Grumari
Centro
 Lapa
Higienópolis
Santa Teresa
 Cosme Velho
Água Santa
 Encantado
Taquara
Vila Kosmos
Vidigal
 São Conrado
Cidade Nova
 Praça da Bandeira
Bonsucesso
Cocotá
 Bancários
Maria da Graça
 Del Castilho
Ribeira
 Cacuia
Lins de Vasconcelos
Engenho Novo
Zumbi
 Pitangueiras
 Praia da Bandeira
Ramos
Engenho de Dentro
Abolição
Deodoro
 Vila Militar
 Campo dos Afonsos
Jardim Sulacap
Oswaldo Cruz
Olaria
Bento Ribeiro
Piedade
Quintino Bocaiúva
Rio Comprido
Praça Seca
Jardim América
Jacaré
 Rocha
 Sampaio
Freguesia
Jardim Carioca
Engenho da Rainha
Brás de Pina
São Cristóvão
 Vasco da Gama
Cascadura
Parque Anchieta
Madur

In [26]:
rj_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barra da Tijuca,-22.99974,-43.365993,Teatro de Câmara,-22.998662,-43.365377,Theater
1,Barra da Tijuca,-22.99974,-43.365993,Galeria Cidade das Artes,-22.999197,-43.366056,Art Gallery
2,Barra da Tijuca,-22.99974,-43.365993,Café das Artes,-22.999408,-43.366605,Café
3,Barra da Tijuca,-22.99974,-43.365993,Cidade das Artes,-22.999604,-43.365989,Music Venue
4,Barra da Tijuca,-22.99974,-43.365993,Cobasi,-22.997846,-43.362360,Pet Store
...,...,...,...,...,...,...,...
3112,Complexo do Alemão,-22.86061,-43.273751,Farmácia Nova Itararé,-22.859702,-43.272017,Pharmacy
3113,Complexo do Alemão,-22.86061,-43.273751,Academia MC Fitness,-22.859442,-43.270659,Gym
3114,Complexo do Alemão,-22.86061,-43.273751,Experimenta Lanches,-22.863348,-43.271767,Fast Food Restaurant
3115,Complexo do Alemão,-22.86061,-43.273751,M&C Fitness,-22.863327,-43.271677,Athletics & Sports


In [27]:
rj_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Alto da Boa Vista,2,2,2,2,2,2
Bancários,24,24,24,24,24,24
Cacuia,3,3,3,3,3,3
Campo dos Afonsos,4,4,4,4,4,4
Cidade Universitária,4,4,4,4,4,4
Cosme Velho,14,14,14,14,14,14
Del Castilho,87,87,87,87,87,87
Encantado,7,7,7,7,7,7
Engenheiro Leal,5,5,5,5,5,5


In [28]:
rj_onehot = pd.get_dummies(rj_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rj_onehot['Neighbourhood'] = rj_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns_rj = [rj_onehot.columns[-1]] + list(rj_onehot.columns[:-1])
rj_onehot = rj_onehot[fixed_columns_rj]

rj_grouped = rj_onehot.groupby('Neighbourhood').mean().reset_index()
rj_grouped

,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,...,Stadium,Stationery Store,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Water Park,Waterfall,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Alto da Boa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bancários,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Cacuia,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Campo dos Afonsos,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Cidade Universitária,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [29]:
# create a new dataframe
neighborhoods_venues_sorted_rj = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_rj['Neighbourhood'] = rj_grouped['Neighbourhood']

for ind in np.arange(rj_grouped.shape[0]):
    neighborhoods_venues_sorted_rj.iloc[ind, 1:] = return_most_common_venues(rj_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_rj

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alto da Boa Vista,Mountain,Scenic Lookout,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio
1,Bancários,Burger Joint,Plaza,Snack Place,Gym / Fitness Center,BBQ Joint,Soup Place,Food,Supermarket,Mini Golf,Seafood Restaurant
2,Cacuia,Nature Preserve,Accessories Store,Thrift / Vintage Store,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
3,Campo dos Afonsos,Athletics & Sports,Museum,Hotel,Sushi Restaurant,Yoga Studio,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
4,Cidade Universitária,Diner,Snack Place,Chinese Restaurant,Dessert Shop,Flower Shop,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
5,Cosme Velho,Café,Train Station,Restaurant,Bakery,Train,Bed & Breakfast,History Museum,Plaza,Art Museum,BBQ Joint
6,Del Castilho,Brazilian Restaurant,Ice Cream Shop,Sporting Goods Shop,Bar,Fast Food Restaurant,Coffee Shop,Steakhouse,Café,Pizza Place,Chocolate Shop
7,Encantado,Athletics & Sports,Bar,Bakery,Gym,Auto Garage,Plaza,Farmers Market,Yoga Studio,Film Studio,Fish & Chips Shop
8,Engenheiro Leal,Bar,Music Venue,Snack Place,Lottery Retailer,Samba School,Fish Market,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop
9,Gamboa,Factory,Brazilian Restaurant,History Museum,Bus Station,Mountain,Cable Car,Fast Food Restaurant,BBQ Joint,Samba School,Stadium


In [30]:
# set number of clusters
kclusters = 5

rj_grouped_clustering = rj_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rj_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 4, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 0, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 3,
       3, 0, 1, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0,
       3, 0, 3, 3, 0, 3, 3, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted_rj.insert(0, 'Cluster Labels', kmeans.labels_)

In [32]:
rj_merged = new_rj.rename(columns={"Neighbourhood":"Neighbourhood"})

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
rj_merged = rj_merged.join(neighborhoods_venues_sorted_rj.set_index('Neighbourhood'), on='Neighbourhood')

rj_merged=rj_merged.dropna()
rj_merged["Cluster Labels"] = rj_merged["Cluster Labels"].astype("int")
rj_merged

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barra da Tijuca,-22.999740,-43.365993,3,Museum,Theater,Farmers Market,Music Venue,Bar,Pet Store,Tea Room,Café,Park,Thai Restaurant
1,Joá,-23.007780,-43.287234,0,Scenic Lookout,Concert Hall,Tunnel,Hotel,Movie Theater,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
2,Gávea,-22.981424,-43.238324,3,Park,Brazilian Restaurant,History Museum,Nightlife Spot,Breakfast Spot,Bar,Bakery,Trail,College Quad,Athletics & Sports
3,Leblon,-22.983556,-43.224938,3,Bar,Brazilian Restaurant,Steakhouse,Gym / Fitness Center,Italian Restaurant,French Restaurant,Sandwich Place,Café,Pizza Place,Cocktail Bar
4,Jardim Guanabara,-22.812836,-43.200779,3,Salad Place,Plaza,Gym,Bar,Bakery,BBQ Joint,Flower Shop,Bed & Breakfast,Deli / Bodega,Garden
5,Ipanema,-22.983956,-43.202216,0,Bar,Food Stand,Pizza Place,Japanese Restaurant,Brazilian Restaurant,Spa,Gym / Fitness Center,Chocolate Shop,Mexican Restaurant,Hostel
6,Lagoa,-22.962466,-43.202488,0,Gym / Fitness Center,Convenience Store,French Restaurant,Bar,Brazilian Restaurant,Bakery,Snack Place,Supermarket,Fast Food Restaurant,Farmers Market
7,Flamengo,-22.933984,-43.174574,0,Gym / Fitness Center,Brazilian Restaurant,Bar,Ice Cream Shop,Bakery,Track,Japanese Restaurant,Supermarket,Sandwich Place,Bookstore
8,Humaitá,-22.954641,-43.200480,3,Bar,Italian Restaurant,Plaza,Pizza Place,Pie Shop,Deli / Bodega,Gift Shop,Snack Place,Farmers Market,Martial Arts School
9,Laranjeiras,-22.934317,-43.187816,3,Bar,Gym / Fitness Center,Lounge,Fruit & Vegetable Store,Coffee Shop,Convenience Store,Yoga Studio,Playground,Sports Bar,Sporting Goods Shop


In [33]:

import matplotlib.cm as cm
import matplotlib.colors as colors
rj_clusters = folium.Map(location=[latitude_rj, longitude_rj],zoom_start=11)
# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(rj_merged['Latitude'], rj_merged['Longitude'], rj_merged['Neighbourhood'], rj_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(rj_clusters)
       
rj_clusters

In [45]:
rj_cluster = rj_grouped.join(neighborhoods_venues_sorted_rj.set_index('Neighbourhood'), on='Neighbourhood')


In [46]:
rj_cluster

,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,...,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Water Park,Waterfall,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alto da Boa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,Mountain,Scenic Lookout,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio
1,Bancários,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,Burger Joint,Plaza,Snack Place,Gym / Fitness Center,BBQ Joint,Soup Place,Food,Supermarket,Mini Golf,Seafood Restaurant
2,Cacuia,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,Nature Preserve,Accessories Store,Thrift / Vintage Store,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
3,Campo dos Afonsos,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [47]:
rj_cluster=rj_cluster.drop(rj_cluster.columns[[293,294,295,296,297,298,299,300,301,302,302]], axis=1)

In [48]:
rj_cluster

,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,...,Stationery Store,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Water Park,Waterfall,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels
0,Alto da Boa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,Bancários,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,Cacuia,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,Campo dos Afonsos,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,Cidade Universitária,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [49]:
rj_clustered = rj_cluster.groupby('Cluster Labels').mean().reset_index()

In [50]:
rj_clustered

,Cluster Labels,ATM,Acai House,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,...,Stadium,Stationery Store,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Water Park,Waterfall,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0.000566,0.001443,0.004824,0.000000,0.003366,0.000124,0.001075,0.001016,0.003645,0.000756,0.000768,0.010228,0.000000,0.010304,0.002919,0.011831,0.000566,0.020136,0.015178,0.000566,0.000978,0.000522,0.008179,0.000845,0.001548,0.000455,0.001781,0.000296,0.000116,0.00000,0.002104,0.000672,0.000578,0.000987,0.000000,0.000000,0.001152,0.055203,0.003165,...,0.002270,0.000000,0.002447,0.000133,0.014016,0.000000,0.005822,0.000000,0.000124,0.001195,0.000000,0.000000,0.000364,0.003913,0.000133,0.000000,0.001394,0.00317,0.000566,0.00384,0.000108,0.000389,0.002041,0.000299,0.002025,0.000768,0.023224,0.001605,0.002620,0.000108,0.001834,0.001152,0.000000,0.003582,0.000000,0.001790,0.000000,0.000566,0.000471,0.000108
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0.000000,0.001425,0.000000,0.000302,0.000000,0.000000,0.002270,0.001771,0.000566,0.000702,0.001352,0.008508,0.000675,0.002506,0.000000,0.011436,0.000000,0.059712,0.126039,0.000000,0.000000,0.000175,0.007092,0.000000,0.001462,0.001930,0.004410,0.001170,0.000000,0.00065,0.003411,0.000000,0.001595,0.000741,0.001253,0.001894,0.000797,0.030759,0.003965,...,0.000501,0.000501,0.007329,0.004386,0.002145,0.000175,0.000709,0.000237,0.000000,0.000000,0.000175,0.000175,0.000000,0.002073,0.000000,0.002188,0.004738,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.002423,0.000000,0.001350,0.000000,0.003509,0.000738,0.001123,0.000702,0.000381,0.000000,0.000797,0.000237,0.003304,0.000175,0.001556,0.001221
4,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [89]:
def return_most_common_venues_cluster(row, num_top_venues_cluster):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues_cluster]
import numpy as np
num_top_venues_cluster = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns_cluster = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns_cluster.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns_cluster.append('{}th Most Common Venue'.format(ind+1))

In [93]:
# create a new dataframe
neighborhoods_venues_sorted_rj_clustered = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_rj_clustered['Cluster Labels'] = rj_clustered['Cluster Labels']
neighborhoods_venues_sorted_rj_clustered = neighborhoods_venues_sorted_rj_clustered.drop(neighborhoods_venues_sorted_rj_clustered.columns[[0]], axis=1)
fixed_columns_rj_first = [neighborhoods_venues_sorted_rj_clustered.columns[-1]] + list(neighborhoods_venues_sorted_rj_clustered.columns[:-1])
neighborhoods_venues_sorted_rj_clustered = neighborhoods_venues_sorted_rj_clustered[fixed_columns_rj_first]


for ind in np.arange(rj_clustered.shape[0]):
    neighborhoods_venues_sorted_rj_clustered.iloc[ind, 1:] = return_most_common_venues_cluster(rj_clustered.iloc[ind, :], num_top_venues_cluster)

neighborhoods_venues_sorted_rj_clustered

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Brazilian Restaurant,Restaurant,Pizza Place,Sandwich Place,Snack Place,Gym / Fitness Center,Gym,Food Truck,Burger Joint,Plaza
1,1,Bakery,Pizza Place,Beer Garden,Music Venue,Yoga Studio,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
2,2,Gym,Yoga Studio,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
3,3,Bar,Bakery,Plaza,Burger Joint,Pizza Place,Brazilian Restaurant,Gym,Restaurant,Gym / Fitness Center,Snack Place
4,4,Beach,Yoga Studio,Farm,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [94]:
neighborhoods_sp_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Moema,Furniture / Home Store,Dessert Shop,Japanese Restaurant,Snack Place,Pharmacy,Pizza Place,Plaza,Middle Eastern Restaurant,Restaurant,Burger Joint


So, the Group of Neighbourhoods that more looks like Moema is the Group 3

That Said, one should think about to live somewhere in Cluster 3:

In [105]:
rj_3=rj_merged[(rj_merged["Cluster Labels"] == 3)]
rj_3

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barra da Tijuca,-22.999740,-43.365993,3,Museum,Theater,Farmers Market,Music Venue,Bar,Pet Store,Tea Room,Café,Park,Thai Restaurant
2,Gávea,-22.981424,-43.238324,3,Park,Brazilian Restaurant,History Museum,Nightlife Spot,Breakfast Spot,Bar,Bakery,Trail,College Quad,Athletics & Sports
3,Leblon,-22.983556,-43.224938,3,Bar,Brazilian Restaurant,Steakhouse,Gym / Fitness Center,Italian Restaurant,French Restaurant,Sandwich Place,Café,Pizza Place,Cocktail Bar
4,Jardim Guanabara,-22.812836,-43.200779,3,Salad Place,Plaza,Gym,Bar,Bakery,BBQ Joint,Flower Shop,Bed & Breakfast,Deli / Bodega,Garden
8,Humaitá,-22.954641,-43.200480,3,Bar,Italian Restaurant,Plaza,Pizza Place,Pie Shop,Deli / Bodega,Gift Shop,Snack Place,Farmers Market,Martial Arts School
9,Laranjeiras,-22.934317,-43.187816,3,Bar,Gym / Fitness Center,Lounge,Fruit & Vegetable Store,Coffee Shop,Convenience Store,Yoga Studio,Playground,Sports Bar,Sporting Goods Shop
10,Jardim Botânico,-22.968385,-43.228694,3,Garden,Soccer Field,Bar,Historic Site,Playground,Café,Bakery,Bistro,Botanical Garden,Fountain
18,Méier,-22.902642,-43.281726,3,Bar,Health & Beauty Service,Brazilian Restaurant,Bakery,Snack Place,Middle Eastern Restaurant,Drugstore,Burger Joint,Dance Studio,Plaza
19,Tijuca,-22.933216,-43.238145,3,Pizza Place,Italian Restaurant,Gym,Bakery,Bar,Café,Deli / Bodega,Japanese Restaurant,Gym / Fitness Center,Paper / Office Supplies Store
22,Anil,-22.955813,-43.338412,3,Bar,Pizza Place,Food Truck,Italian Restaurant,BBQ Joint,Electronics Store,Sandwich Place,Farmers Market,Snack Place,Burger Joint


In [106]:
rj_merged[(rj_merged["Cluster Labels"] == 3)].count() 

Neighbourhood             57
Latitude                  57
Longitude                 57
Cluster Labels            57
1st Most Common Venue     57
2nd Most Common Venue     57
3rd Most Common Venue     57
4th Most Common Venue     57
5th Most Common Venue     57
6th Most Common Venue     57
7th Most Common Venue     57
8th Most Common Venue     57
9th Most Common Venue     57
10th Most Common Venue    57
dtype: int64

In [107]:

import matplotlib.cm as cm
import matplotlib.colors as colors
rj_clusters = folium.Map(location=[latitude_rj, longitude_rj],zoom_start=11)
# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(rj_3['Latitude'], rj_3['Longitude'], rj_3['Neighbourhood'], rj_3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(rj_clusters)
       
rj_clusters